In [1]:
import tensorflow as tf
from tensorflow.keras import layers, models, regularizers
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from keras.callbacks import EarlyStopping

import numpy as np

In [2]:
(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()

In [3]:
x_train, x_test = x_train / 255.0, x_test / 255.0

In [4]:
x_train = x_train.reshape((x_train.shape[0], 28, 28, 1))
x_test = x_test.reshape((x_test.shape[0], 28, 28, 1))

In [5]:
mean_value = np.mean(x_train)
std_value = np.std(x_train)

In [6]:
x_train_std = (x_train - mean_value) / std_value
x_test_std = (x_test - mean_value) / std_value

In [7]:
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

In [8]:
x_train_std, x_val_std, y_train, y_val = train_test_split(x_train_std, y_train, test_size=0.2, random_state=2137)

In [9]:
model = models.Sequential()
model.add(layers.experimental.preprocessing.RandomCrop(28, 28, input_shape=(28, 28, 1)))
model.add(layers.experimental.preprocessing.RandomFlip("horizontal"))

model.add(layers.Conv2D(16, (3, 3),
                        activation='relu',
#                         kernel_regularizer=regularizers.l2(0.01),
#                         kernel_initializer = "glorot_normal",
                        padding = "same"))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Conv2D(32, (3, 3),
                        activation='relu',
#                         kernel_regularizer=regularizers.l2(0.01),
#                         kernel_initializer = "glorot_normal",
                        padding = "same"))
model.add(layers.MaxPooling2D((2,2)))

model.add(layers.Flatten())

model.add(layers.Dense(256,
                       activation='relu',
                       kernel_regularizer=regularizers.l2(0.01)))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(10, activation='softmax'))

In [10]:
model.compile(optimizer = "adam",
             loss = "categorical_crossentropy",
             metrics = ["accuracy"],
             )

early_stopping = EarlyStopping(monitor = "val_accuracy", patience = 5, restore_best_weights = True)

In [11]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 random_crop (RandomCrop)    (None, 28, 28, 1)         0         
                                                                 
 random_flip (RandomFlip)    (None, 28, 28, 1)         0         
                                                                 
 conv2d (Conv2D)             (None, 28, 28, 16)        160       
                                                                 
 max_pooling2d (MaxPooling2  (None, 14, 14, 16)        0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 14, 14, 32)        4640      
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 7, 7, 32)          0         
 g2D)                                                   

In [12]:
model.fit(x_train_std, y_train, epochs = 100, validation_data = (x_val_std, y_val), batch_size = 512, callbacks=[early_stopping])
test_loss, test_acc = model.evaluate(x_test_std, y_test)
print(f'ACCURACY: {test_acc * 100:.2f}%')

Epoch 1/100
94/94 [==============================] - 9s 82ms/step - loss: 2.3652 - accuracy: 0.6998 - val_loss: 0.9134 - val_accuracy: 0.8209
Epoch 2/100
94/94 [==============================] - 8s 80ms/step - loss: 0.7702 - accuracy: 0.8159 - val_loss: 0.5915 - val_accuracy: 0.8542
Epoch 3/100
94/94 [==============================] - 7s 79ms/step - loss: 0.6113 - accuracy: 0.8342 - val_loss: 0.5083 - val_accuracy: 0.8650
Epoch 4/100
94/94 [==============================] - 7s 76ms/step - loss: 0.5568 - accuracy: 0.8426 - val_loss: 0.4745 - val_accuracy: 0.8724
Epoch 5/100
94/94 [==============================] - 8s 80ms/step - loss: 0.5315 - accuracy: 0.8498 - val_loss: 0.4692 - val_accuracy: 0.8748
Epoch 6/100
94/94 [==============================] - 7s 78ms/step - loss: 0.5105 - accuracy: 0.8555 - val_loss: 0.4464 - val_accuracy: 0.8775
Epoch 7/100
94/94 [==============================] - 8s 84ms/step - loss: 0.4949 - accuracy: 0.8580 - val_loss: 0.4529 - val_accuracy: 0.8729
Epoch 

KeyboardInterrupt: 